In [2]:
# standard python tools
import numpy as np
from glob import glob

In [3]:
# make sure pyqae is available
import pyqae as pq
    
# check to see if it is running on pyspark (if not use the simplespark backend)
try:
    import pyspark
except ImportError:
    print('Using a fake spark backend for processing')

    import pyqae.simplespark as pyspark
    sc = pyspark.LocalSparkContext()

from pyqae.images import viz
# lazy image 
from pyqae.images.lazy import paths_to_tiled_image, DiskMappedLazyImage, backends

In [4]:
base_path = "/Users/mader/Dropbox/4Quant/Projects/TumorSegmentation/10092825/"
little_image_list = glob(os.path.join(base_path,"0004_t1_tse_sag_384/*.tif"))
big_image_list = ['/Volumes/ExDisk/cor03639_CH2.tif']*5
really_big_image_list = ['/Volumes/TOSHIBA EXT/1616.tif']

In [5]:
start_img = DiskMappedLazyImage(big_image_list[0], backends[1])
print(start_img.shape)

(22902, 30021)


In [50]:
def create_line_break(x_tile_size, x_dim):
    return [(x, min(x+x_tile_size, x_dim)) for x in range(0, x_dim, x_tile_size)]

def create_tiles(x_tile_size, x_dim, y_tile_size, y_dim):
    return [(x_tile, y_tile) for x_tile in create_line_break(x_tile_size, x_dim) 
            for y_tile in create_line_break(y_tile_size, y_dim)]
def tile_dict(tile_list):
    return [{'x_start': int(xs), 'y_start': int(ys), 
             'x_end': int(xe), 'y_end': int(ye)} for ((xs, xe), (ys, ye)) in tile_list]

In [43]:
print(create_line_break(256, 1025))

[(0, 256), (256, 512), (512, 768), (768, 1024), (1024, 1025)]


In [48]:
im_size = np.random.randint(1000, 4000, size = 2)
for tile_size in np.random.randint(100, 300, size = (3,2)):
    c_tiles = create_tiles(tile_size[0], im_size[0],tile_size[1], im_size[1])
    tile_sum = np.sum([(xe-xs)*(ye-ys) for ((xs, xe), (ys, ye)) in c_tiles])
    print(np.random.permutation(c_tiles)[0:2])
    assert tile_sum == im_size[0]*im_size[1], "Total pixel count cannot change, {} != {}".format(im_size[0]*im_size[1], tile_sum)

[[[   0  276]
  [ 804 1005]]

 [[   0  276]
  [1206 1407]]]
[[[ 600  800]
  [ 652  815]]

 [[1800 2000]
  [ 815  978]]]
[[[1110 1332]
  [ 362  543]]

 [[1332 1554]
  [ 181  362]]]


In [51]:
sqlContext.createDataFrame(tile_dict(c_tiles))

/Volumes/ExDisk/spark-2.0.0-bin-hadoop2.7/python/pyspark/sql/session.py:316: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


DataFrame[x_end: bigint, x_start: bigint, y_end: bigint, y_start: bigint]